In [ ]:
from ultralytics import YOLO
from datetime import datetime

In [ ]:
# Load YOLOv11 model configuration
model = YOLO("yolo11s.pt").to("cuda")

In [ ]:
# Prepare training arguments
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_dir = f"runs/train/{timestamp}"
train_args = {
    "data": "../data/dataset/data.yaml",
    "epochs": 150,
    "imgsz": 1280,
    "batch": 8,
    "patience": 50,
    "save": True,
    "optimizer": "AdamW",
    "save_dir": save_dir,
    "lr0": 0.01,
    "close_mosaic": 20,
    "name": f"train_{timestamp}"
}

In [ ]:
# Train model on custom dataset
results = model.train(**train_args)

In [ ]:
metrics = model.val()
print(metrics)

In [ ]:
# Print training results
print(results)

## Export to TensorRT

In [ ]:
# Optional: LOAD model
model = YOLO("runs/detect/720_best/weights/best.pt").to("cuda")

In [ ]:
model.export(
    format="engine",           
    half=True,                
    imgsz=1280,                    
    batch=6,                   
    dynamic=True,             
)